In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
url="https://search.hankyung.com/search/news?query=%EA%B8%88%EB%A6%AC&sort=RANK%2FDESC%2CDATE%2FDESC&period=DATE&area=ALL&sdate=2008.01.01&edate=2015.01.01&exact=%EA%B8%88%EB%A6%AC&include=&except=&hk_only=n&page=1"

In [ ]:
response = requests.get(url, headers={}, data={})
print(response.status_code) # 200 = 정상
print(response.text) # HTML 

### 링크 따 코드

In [ ]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import csv

# 🔹 검색 결과 페이지 URL (페이지 번호 제외)
base_url = 'https://search.hankyung.com/search/news?query=%EA%B8%88%EB%A6%AC&sort=RANK%2FDESC%2CDATE%2FDESC&period=DATE&area=ALL&sdate=2010.01.01&edate=2025.02.27&exact=%EA%B8%88%EB%A6%AC&include=&except=%EA%B3%84%EC%A2%8C%2C%EC%9B%94%EB%93%9C%EC%BB%B5%2C%5B%ED%8F%AC%ED%86%A0%5D%2C%5B%ED%8E%80%EB%93%9C%EB%A7%A4%EB%8B%88%EC%A0%80%5D%2C%EC%83%81%EB%8B%B4%2C%EC%B6%94%EC%B2%9C&hk_only=y&page={}'  # 페이지 번호 포맷 수정

headers = {'User-Agent': 'Mozilla/5.0'}
all_links = []  # 모든 기사 링크를 저장할 리스트

# 🔹 페이지 크롤링 함수 정의
def crawl_page(page):
    global all_links
    search_url = base_url.format(page)  # 페이지 번호 포맷팅
    try:
        res = requests.get(search_url, headers=headers, timeout=5)
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, 'html.parser')

            # 🔹 기사 제목과 링크 가져오기
            articles = soup.find_all('em', class_='tit')
            if articles:
                for article in articles:
                    link = article.find_parent('a')['href']
                    all_links.append(link)

            print(f"페이지 {page} 크롤링 완료. 현재까지 수집된 기사 수: {len(all_links)}")

        else:
            print(f"페이지 {page} 요청 실패")
    except requests.exceptions.RequestException as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

# 🔹 멀티스레딩으로 페이지 크롤링
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # 페이지 번호 1부터 8589까지
    page_numbers = range(1, 8571)  # 8589까지 크롤링
    executor.map(crawl_page, page_numbers)

# 🔹 CSV 파일로 저장
csv_filename = 'news_links2.csv'
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Link"])  # CSV 파일의 헤더 작성

    for link in all_links:
        writer.writerow([link])  # 각 링크를 CSV 파일에 기록

print(f"\n🔹 CSV 파일 '{csv_filename}'에 {len(all_links)}개의 링크를 저장했습니다.")


In [19]:
import pandas as pd

# CSV 파일을 데이터프레임으로 읽어오기
df = pd.read_csv('news_links.csv')

# 데이터프레임의 처음 5개 행을 출력하여 확인
print(df.head())


                                             Link
0  https://www.hankyung.com/article/2016080193475
1  https://www.hankyung.com/article/2016080192518
2  https://www.hankyung.com/article/2016080192308
3  https://www.hankyung.com/article/2016080190768
4  https://www.hankyung.com/article/2016080191425


### 링크 타고 들어가 기사 갖고 오기 title , contents , date 순으로

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# 🔹 CSV 파일에서 기사 링크 읽기
csv_file = 'news_links2.csv'
df = pd.read_csv(csv_file, header=None)

# 🔹 CSV 데이터 확인 (디버깅용)
print("🔍 CSV 데이터 프리뷰:")
print(df.head())  # 처음 5행 출력
print("🔍 열 인덱스:", df.columns.tolist())  # 열 번호 확인

# 🔹 기사 링크 리스트 (테스트용으로 처음 5개만 사용)
# 열이 하나しかない 경우 0, 두 번째 열을 원하면 1로 설정
links_column = 0  # ★ 여기를 CSV에 맞게 수정 (0 또는 1 등)
links = df[links_column].tolist()[10001:83540]  # 처음 5개만 슬라이싱

# 🔹 불필요한 기자 정보 제거 함수
def clean_text(text):
    text = re.sub(r"([가-힣]{2,4})\s?(기자|특파원)", "", text)
    text = re.sub(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", "", text)
    text = re.sub(r"\(.*?\)", "", text)
    text = re.sub(r"\[.*?\]", "", text)
    return text.strip()

# 🔹 기사 크롤링 함수
def scrape_article(link):
    try:
        res = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
        if res.status_code != 200:
            print(f"❌ 요청 실패: {res.status_code} - {link}")
            return None

        soup = BeautifulSoup(res.text, 'html.parser')

        # ✅ 제목 가져오기
        title = soup.select_one("h1.headline")
        title = title.text.strip() if title else "제목 없음"

        # ✅ 날짜 가져오기
        date = soup.select_one("span.txt-date")
        date = date.text.strip() if date else "날짜 없음"
        date = re.sub(r'\d{2}:\d{2}', '', date).strip()
        date_match = re.search(r"(\d{4})년 (\d{1,2})월 (\d{1,2})일", date)
        if date_match:
            date = f"{date_match.group(1)}-{date_match.group(2).zfill(2)}-{date_match.group(3).zfill(2)}"

        # ✅ 본문 가져오기
        article_body = soup.find("div", class_="article-body", id="articletxt")
        if article_body:
            contents = "\n".join([element for element in article_body.stripped_strings])
            contents = clean_text(contents)
        else:
            contents = "본문 없음"

        return {'title': title, 'contents': contents, 'date': date}
    
    except Exception as e:
        print(f"🚨 오류 발생: {link} - {e}")
        return None

# 🔹 멀티스레딩 크롤링 실행 (테스트용)
def main():
    print(f"🔍 테스트: {len(links)}개의 기사 크롤링 시작...")

    results = []
    with ThreadPoolExecutor(max_workers=15) as executor:
        futures = {executor.submit(scrape_article, link): link for link in links}
        for future in as_completed(futures):
            result = future.result()
            if result:
                results.append(result)
                print(f"✅ 크롤링 완료: {result['title']}")

    # 🔹 크롤링 결과를 DataFrame으로 변환 후 CSV 저장
    output_df = pd.DataFrame(results, columns=['title', 'contents', 'date'])
    output_df.to_csv('test_scraped_articles2.csv', index=False, encoding='utf-8-sig')

    print("📁 테스트 완료! 'test_scraped_articles2.csv' 파일 저장 완료.")

if __name__ == '__main__':
    main()

### 개행 문제 해결 코드

In [25]:
import pandas as pd

# 🔹 기존 CSV 파일 읽기
input_csv_file = 'test_scraped_articles.csv'
df = pd.read_csv(input_csv_file)

# 🔹 제목이 '제목 없음'인 행 제거
df_filtered = df[df['title'] != '제목 없음']

# 🔹 결과를 새로운 CSV 파일로 저장
output_csv_file = 'filtered_articles_real.csv'
df_filtered.to_csv(output_csv_file, index=False, encoding='utf-8-sig')

print(f"🎉 '제목 없음'이 제거된 CSV 파일이 '{output_csv_file}'로 저장되었습니다.")


🎉 '제목 없음'이 제거된 CSV 파일이 'filtered_articles_real.csv'로 저장되었습니다.


In [26]:
import pandas as pd
import csv  # CSV 옵션 추가

# 🔹 기존 CSV 파일 읽기
input_csv_file = 'filtered_articles_real.csv'
df = pd.read_csv(input_csv_file)

# 🔹 CSV 저장 시 개행 문제 해결
output_csv_file = 'filtered_articles_fixed.csv'
df.to_csv(output_csv_file, index=False, encoding='utf-8-sig', lineterminator='\n', quoting=csv.QUOTE_MINIMAL)

print(f"🎉 개행 문제 해결! 새로운 파일: '{output_csv_file}'")


🎉 개행 문제 해결! 새로운 파일: 'filtered_articles_fixed.csv'


In [27]:
df.to_csv(output_csv_file, index=False, encoding='utf-8-sig', lineterminator='\n', quoting=csv.QUOTE_ALL)


In [28]:
df["contents"] = df["contents"].astype(str).replace("\n", " ", regex=True)
df.to_csv(output_csv_file, index=False, encoding='utf-8-sig', lineterminator='\n')


### 기사 정제 코드

In [59]:
import pandas as pd

# 🔹 CSV 파일 읽기 (열 이름 없이 읽기)
csv_file = '20.csv'  # CSV 파일 경로
df = pd.read_csv(csv_file, header=None)  # header=None으로 열 이름 없이 읽기

# 🔹 키워드 리스트 (기존 키워드 + 추가 키워드)
keywords = [
    "임영웅", "티몬", "위메프", "티메프", "앱", "낱말", "계좌", "달빛", "적금", "아이폰", "tkg", "역세권", "토스", "학원", "농협", "재개발", 
    "비트코인", "공인", "부산은행", "앱", "배달", "건강", "보험", "분양", "입주", "철도", "AI", "네이버", "한샘", "암호", "코인", "현대", 
    "삼성", "중고차", "피아노", "백화점", "볼보", "위스키", "카드", "패딩", "변신", "투자", "스탁론", "카뱅", "다이소", "멤버십", "물량", 
    "루이비통", "증권", "수익", "skt", "kt", "신한", "우리", "하나", "출시", "현대차", "강남", "콘서트", "건설조합", "캠페인", "요기요", 
    "배달", "폭우", "표창", "자연재해", "lg유플러스", "가짜", "사료값", "보이스", "맨해튼", "블로그", "온택트", "박람회", "제네시스", 
    "기아", "신차", "오피스텔", "스마트폰", "esg", "이민", "웰컴저축", "장병", "둔촌", "주공", "저탄소", "킨텍스", "스타트업", "lh", 
    "롯데", "언더아머", "케이뱅크", "네이버페이", "판매", "퍼즐", "다올", "선착순", "출시", "토지거래허가", "가스공급", "sbi", "입점", 
    "페이코", "경품", "에어팟", "골드바", "월세", "르노", "수협", "축협", "신혼", "타운", "우대형", "다방", "무료조회", "우리카드", 
    "여행", "저소득층", "임대주택", "급전", "실종", "여대생", "남자친구", "소재", "아파트", "핀테크",  "게임", "영화", "드라마", "예능", "음악", "콘서트", "팬미팅", "유튜브", "인플루언서", "패션", "뷰티", "메이크업", "헤어", "스타일", 
    "운동", "헬스", "요가", "필라테스", "등산", "골프", "축구", "야구", "농구", "테니스", "스키", "스노우보드", "여행", "맛집", "카페", 
    "디저트", "와인", "맥주", "칵테일", "커피", "차", "반려동물", "강아지", "고양이", "애완동물", "동물병원", "동물원", "공원", "전시회", 
    "박물관", "미술관", "공연", "뮤지컬", "연극", "오페라", "발레", "클래식", "재즈", "락", "힙합", "K-pop", "아이돌", "배우", "가수", 
    "작가", "화가", "사진", "촬영", "드론", "VR", "AR", "메타버스", "게임", "e스포츠", "프로게이머", "스마트폰", "태블릿", "노트북", 
    "데스크탑", "프린터", "카메라", "스피커", "이어폰", "스마트워치", "가전제품", "냉장고", "세탁기", "에어컨", "청소기", "로봇", "드론", 
    "스마트홈", "IoT", "인공지능", "머신러닝", "딥러닝", "블록체인", "가상화폐", "NFT", "메타버스", "스타트업", "창업", "벤처", "투자", 
    "크라우드펀딩", "쇼핑", "온라인쇼핑", "오프라인쇼핑", "할인", "이벤트", "경품", "추첨", "공모전", "대회", "수상", "시상식", "연예인", 
    "유명인", "인스타그램", "페이스북", "트위터", "SNS", "소셜미디어", "블로그", "유튜버", "스트리머", "방송", "라디오", "팟캐스트", 
    "웹툰", "만화", "애니메이션", "영화제", "영화평론", "영화리뷰", "드라마리뷰", "예능리뷰", "음악리뷰", "책", "소설", "시", "에세이", 
    "독서", "서점", "도서관", "학습", "교육", "온라인강의", "오프라인강의", "학원", "과외", "유학", "어학연수", "외국어", "영어", 
    "일본어", "중국어", "프랑스어", "스페인어", "독일어", "이탈리아어", "러시아어", "아랍어", "힌디어", "포르투갈어", "네덜란드어", 
    "스웨덴어", "노르웨이어", "덴마크어", "핀란드어", "그리스어", "터키어", "베트남어", "태국어", "인도네시아어", "말레이시아어", 
    "필리핀어", "싱가포르어", "홍콩어", "대만어", "한국어", "한글", "한국문화", "K-pop", "K-drama", "K-beauty", "K-food", "한식", 
    "김치", "불고기", "비빔밥", "떡볶이", "라면", "삼겹살", "갈비", "족발", "보쌈", "냉면", "순대", "떡", "한과", "전통음식", "전통문화", 
    "전통의상", "한복", "전통춤", "전통음악", "사물놀이", "판소리", "민요", "전통악기", "가야금", "해금", "거문고", "대금", "소금", 
    "피리", "장구", "북", "징", "꽹과리", "소고", "풍물", "농악", "민속놀이", "윷놀이", "널뛰기", "제기차기", "연날리기", "씨름", 
    "태권도", "합기도", "유도", "검도", "궁도", "전통무술", "전통예술", "전통공예", "도자기", "청자", "백자", "분청사기", "도예", 
    "목공예","수지","mc몽","혁오","김희선","예능","영업익","옥타곤","지드래곤","유이","엔딩","배수정","배용준","kbs","삼디","김혜자","유재석","아이유","예능","이동욱","드라마","배성재","아나운서","sbs","mbc","별에서","한그루"
     "박명수","겨울방학","여드름","유아인","비타민","셀카","베이글","박신혜","장수원","연인","촬영","흥행","예원","정준하","김수현","이영자","JYJ","김성균","이종석","셀카가","우승","사과","제약","별그대","SM","미녀"
      "임수향","지진희","LG유플러스","올림픽","이규혁","신대철","엑소","김경란","김태희","슈퍼맨","진짜사나이","로또","송재림","고아성","혜령","국카스텐","심금","시청률","히든싱어","비염","임신","TV", "가족끼리","성탄절"," 나나","`전설의 마녀`",
       "‘진짜 사나이’","`런닝맨`","송일국","`힐링캠프`","열애","`미생`","`죽기전에","죽기 전에" ,"로맨스의","우지원-이교영","신해철","`뻐꾸기 둥지`"# 추가 키워드
]

# 🔹 키워드 필터링 함수
def contains_keyword(text, keywords):
    """
    주어진 텍스트에 키워드가 포함되어 있는지 확인합니다.
    """
    if pd.isna(text):  # NaN 값 처리
        return False
    return any(keyword in text for keyword in keywords)

# 🔹 필터링된 링크 추출
# 제목(0번 열)에서 키워드가 포함된 행 제외
filtered_df = df[~df[0].apply(lambda x: contains_keyword(x, keywords))]

# 🔹 결과 확인
print(f"원본 데이터 개수: {len(df)}")
print(f"필터링 후 데이터 개수: {len(filtered_df)}")

# 🔹 필터링된 데이터 저장
filtered_csv_file = '20.csv'  # 저장할 파일 경로
filtered_df.to_csv(filtered_csv_file, index=False, header=False)  # header=False로 열 이름 없이 저장
print(f"필터링된 데이터를 '{filtered_csv_file}' 파일로 저장했습니다.")

원본 데이터 개수: 9026
필터링 후 데이터 개수: 9021
필터링된 데이터를 '20.csv' 파일로 저장했습니다.
